## Requirements

In [4]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import spacy
from torch.nn.utils.rnn import pad_sequence
from torch.nn.data import DataLoader, Dataset
from PIL import Image

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

ModuleNotFoundError: No module named 'torch.nn.data'

In [2]:
pip install spacy


  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.4
    Uninstalling colorama-0.4.4:
      Successfully uninstalled colorama-0.4.4


## Aladdin Pearson

In [ ]:
class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0:"<PAD>",1:"<SOS>",2:"<EOS>",3:"<UNK>"}
        self.stoi = {"<PAD>":0,"<SOS>":1,"<EOS>":2,"<UNK>":3}
        self.freq_threshold = freq_threshold
    
    def __len__(self):
        return len(self.itos)
    
    @staticmethod
    def tokenize_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]
    
    def build_vocabulary
        
        

In [ ]:
class aksharantar(Dataset):
    def __init__(self, root_dir, data_file, transform=None, freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(data_file)
        self.transform = transform
        
        # Get the latin and devangari script
        self.latin = self.df.iloc[:,0]
        self.hindi = self.df.iloc[:,1]
        
        # Initialize vocabulary and build vocab
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.hindi.tolist())
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        latin = self.latin[index]
        hindi = self.hindi[index]
        
#         if self.transform is not None:
#             hindi = self.transform(hindi)
        
        numericalized_hindi = [self.vocab.stoi["<SOS>"]]
        numericalized_hindi += self.vocab.numericalize(hindi)
        numericalized_hindi.append(self.vocab.stoi["<EOS"])
        
        numericalized_latin = [self.vocab.stoi["<SOS>"]]
        numericalized_latin += self.vocab.numericalize(latin)
        numericalized_latin.append(self.vocab.stoi["<EOS"])
        
        return torch.tensor(numericalized_latin), torch.tensor(numericalized_hindi) 
               
        

## Loading the data and preprocessing

In [152]:
# Loading the dataset
# Give the path of the downloaded datatset for your model
train_path = r"C:\Users\HICLIPS-ASK\aksharantar_sampled\hin\hin_train.csv"
val_path = r"C:\Users\HICLIPS-ASK\aksharantar_sampled\hin\hin_valid.csv"
test_path = r"C:\Users\HICLIPS-ASK\aksharantar_sampled\hin\hin_test.csv"
train_data = pd.read_csv(train_path,header=None)
valid_data = pd.read_csv(val_path,header=None)
test_data = pd.read_csv(test_path,header=None)

In [153]:
train_data

,0,1
0,shastragaar,शस्त्रागार
1,bindhya,बिन्द्या
2,kirankant,किरणकांत
3,yagyopaveet,यज्ञोपवीत
4,ratania,रटानिया
...,...,...
51195,toned,टोंड
51196,mutanaazaa,मुतनाज़ा
51197,asahmaton,असहमतों
51198,sulgaayin,सुलगायीं


In [266]:
def prepare_data(data):
    """
    This function will convert our 'data' into input(x) and labels(y)
    Latin word taken as x for the modelling purpose
    devanagri word taken as y for the modelling purpose
    
    """
    x_data = data.iloc[:,0].values
    y_data = data.iloc[:,1].values
    
    return x_data, y_data

In [267]:
x_train, y_train = prepare_data(train_data)
x_valid, y_valid = prepare_data(valid_data)
x_test, y_test = prepare_data(test_data)

In [234]:
def letter2index_index2letter(alpha_type):
    """
    alpha_type: it takes 'english' or 'hindi' value
    This function will create dictionaies of character:index(key:value) for both the language scripts
    
    """
    
    if alpha_type == 'english':
        alphabet_eng = 'abcdefghijklmnopqrstuvwxyz'
        letter2index_eng = {}
        letter2index_eng['-sos-'] = 0

        for ind, c in enumerate(alphabet_eng):
            letter2index_eng[c] = ind+1
        letter2index_eng['-eos-'] = len(letter2index_eng)
        
        print("__letter to index__")
        print(letter2index_eng)

        print(" ")
        print("__Index to letter__")
        
        ### Index to Letter
        index2letter_eng = {}
        index2letter_eng[0] = '-sos-'
        for ind, c in enumerate(alphabet_eng):
            index2letter_eng[ind+1] = c
        index2letter_eng[len(index2letter_eng)] = '-eos-'
        print(index2letter_eng)
        
        return letter2index_eng, index2letter_eng
        
    if alpha_type == 'hindi':
        alphabet_hin = [chr(alpha) for alpha in range(2304, 2432)]
        letter2index_hin = {}
        letter2index_hin['-sos-'] = 0

        for ind, c in enumerate(alphabet_hin):
            letter2index_hin[c] = ind+1
        letter2index_hin['-eos-'] = len(letter2index_hin)
        
        print("__letter to index__")
        print(letter2index_hin)

        print(" ")
        print("__Index to letter__")
        
        ### Index to Letter
        index2letter_hin = {}
        index2letter_hin[0] = '-sos-'
        for ind, c in enumerate(alphabet_hin):
            index2letter_hin[ind+1] = c
        index2letter_hin[len(index2letter_hin)] = '-eos-'
        print(index2letter_hin)
        
        return letter2index_hin, index2letter_hin

In [235]:
letter2index_hin, index2letter_hin = letter2index_index2letter('hindi')
print(" ")
letter2index_eng, index2letter_eng = letter2index_index2letter('english')

__letter to index__
{'-sos-': 0, 'ऀ': 1, 'ँ': 2, 'ं': 3, 'ः': 4, 'ऄ': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ऌ': 13, 'ऍ': 14, 'ऎ': 15, 'ए': 16, 'ऐ': 17, 'ऑ': 18, 'ऒ': 19, 'ओ': 20, 'औ': 21, 'क': 22, 'ख': 23, 'ग': 24, 'घ': 25, 'ङ': 26, 'च': 27, 'छ': 28, 'ज': 29, 'झ': 30, 'ञ': 31, 'ट': 32, 'ठ': 33, 'ड': 34, 'ढ': 35, 'ण': 36, 'त': 37, 'थ': 38, 'द': 39, 'ध': 40, 'न': 41, 'ऩ': 42, 'प': 43, 'फ': 44, 'ब': 45, 'भ': 46, 'म': 47, 'य': 48, 'र': 49, 'ऱ': 50, 'ल': 51, 'ळ': 52, 'ऴ': 53, 'व': 54, 'श': 55, 'ष': 56, 'स': 57, 'ह': 58, 'ऺ': 59, 'ऻ': 60, '़': 61, 'ऽ': 62, 'ा': 63, 'ि': 64, 'ी': 65, 'ु': 66, 'ू': 67, 'ृ': 68, 'ॄ': 69, 'ॅ': 70, 'ॆ': 71, 'े': 72, 'ै': 73, 'ॉ': 74, 'ॊ': 75, 'ो': 76, 'ौ': 77, '्': 78, 'ॎ': 79, 'ॏ': 80, 'ॐ': 81, '॑': 82, '॒': 83, '॓': 84, '॔': 85, 'ॕ': 86, 'ॖ': 87, 'ॗ': 88, 'क़': 89, 'ख़': 90, 'ग़': 91, 'ज़': 92, 'ड़': 93, 'ढ़': 94, 'फ़': 95, 'य़': 96, 'ॠ': 97, 'ॡ': 98, 'ॢ': 99, 'ॣ': 100, '।': 101, '॥': 102, '०': 103, '१': 104, '२': 105, '३': 106, '४': 107, '५': 

In [320]:
# encoding the words into vector form
def word2vector(word, letter2index_dict):
    """
    word: the word which needed to be converted into vector
    letter2index_dict: Dictionary created to map the letter to index
    This function convert the words into a vector according to the indexing done above.
    This will help to make word compatiable for the training
    
    """
    
    encoded_vector = []  
    encoded_vector.append(letter2index_dict['-sos-'])
    for i, w in enumerate(word):
        encoded_vector.append(letter2index_dict[w])
    
    encoded_vector.append(letter2index_dict['-eos-'])
    return encoded_vector


In [322]:
encoded_vector = word2vector("बिन्द्या", letter2index_hin)
encoded_vector

[0, 45, 64, 41, 78, 39, 78, 48, 63, 129]

In [345]:
def process_words(data, letter2index_dict, word2vector):
    """
    word: the word which needed to be converted into vector.
    letter2index_dict: Give the Dictionary created to map the letter to index
    for particular language.
    word2vector: is a function to convert words into numerical vector
    This function will convert first the words into a vecotr form then change the length of all the vectors same as that of 
    longest lengrh vector. Will return a torch tensor for further processing
    
    """


    # Create an empty list to store the encoded representations of words
    temp = []  
    # Variable to keep track of the maximum length of words
    max_length = 0  

    # Iterate over each word in x_train along with its index
    for i, word in enumerate(data):
        rep = word2vector(word, letter2index_dict)  # Encode the word using the function 
        temp.append(rep)  # Add the encoded representation to temp list

        # Check if the current word is longer than the previous maximum length
        if max_length < len(rep):
            max_length = len(rep)  # Update the maximum length
            index = i  # Store the index of the word with maximum length
    
    #print(f"the max word size for {data} is: ",max_length)
    vectors = []  # Create an empty list to store the final encoded representations of words

    # Iterate over each encoded representation in temp
    for word in temp:
        # Append zeros to the encoded representation if its length is shorter than the maximum length
        word = word + ([0] * (max_length - len(word)))
        vectors.append(word)  # Add the updated encoded representation to vectors
    
    # Transforming the list into pytorch tensor
    vectors = torch.tensor(vectors, dtype=torch.long, device=device)

    return vectors


In [362]:
x_train, y_train = process_words(x_train, letter2index_eng,word2vector), process_words(y_train, letter2index_hin,word2vector)
x_valid, y_valid = process_words(x_valid, letter2index_eng,word2vector), process_words(y_valid, letter2index_hin,word2vector)
x_test, y_test = process_words(x_test, letter2index_eng,word2vector), process_words(y_test, letter2index_hin,word2vector)

In [ ]:
## for english
# 26 51200
# 22 51200
# 24 4096
# 22 4096
# 28 4096
# 22 4096
## for hindi

In [240]:
embedded = nn.Embedding(x_train_vector.size(0), 32).to(device)
out = embedded(x_train_vector)

In [241]:
out.size()

torch.Size([51200, 24, 32])

In [243]:
hidden = EncoderRNN.initHidden()

TypeError: initHidden() missing 1 required positional argument: 'self'

In [242]:
output, hidden = nn.GRU(out, hidden).to(device)

NameError: name 'hidden' is not defined

In [193]:
#We’ll need a unique index per word to use as the inputs and targets of the networks later. 
#To keep track of all this we will use a helper class called Lang which has word → index (word2index)
#and index → word (index2word) dictionaries, as well as a count of each word word2count
#which will be used to replace rare words later.

# SOS_token = 0
# EOS_token = 1


# class Lang:
#     def __init__(self, name):
#         self.name = name
#         self.characters2index = {}
#         self.characters2count = {}
#         self.index2characters = {0: "SOS", 1: "EOS"}
#         self.n_characters = 2  # Count SOS and EOS

#     def addWord(self, word):
#         for character in list(word):
#             self.addCharacter(character)

#     def addCharacter(self, character):
#         if character not in self.characters2index:
#             self.characters2index[character] = self.n_characters
#             self.characters2count[character] = 1
#             self.index2characters[self.n_characters] = character
#             self.n_characters += 1
#         else:
#             self.characters2count[character] += 1

In [194]:
# input_lang = Lang('eng')
# output_lang = Lang('devanagari')

In [195]:
# for pair in zip(give_pair(train_data,0),give_pair(train_data,1)):  
#     input_lang.addWord(pair[0])
#     output_lang.addWord(pair[1])
# print("Counted words:")
# print(input_lang.name, input_lang.n_characters)
# print(output_lang.name, output_lang.n_characters)

## The Seq2Seq Model

### The Encoder

In [446]:
class EncoderRNN(nn.Module):
    #input_size represents the dimensionality of the 
    #encoder's input space, indicating the number of possible input tokens or
    #categories that the coder can generate.
    
    def __init__(self, input_size, hidden_size, cell_type):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size)

    def forward(self, input, hidden, cell_type):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        if cell_type == 'gru':
            output, hidden = self.gru(output, hidden)
        if cell_type == 'lstm':
            output, hidden = self.lstm(output, hidden)
        if cell_type == 'rnn':
            output, hidden = self.rnn(output, hidden)
        
        return output, hidden
    
    # This method is called at the beginning of each new input sequence
    # to reset the hidden state.
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [352]:
# hidden: This argument represents the initial hidden state of the RNN. It is expected to be a tensor of
# shape (num_layers * num_directions, batch_size, hidden_size). Since we are dealing with a single-layer RNN, 
# the tensor shape is (1, 1, hidden_size). The hidden tensor is updated during the forward pass and passed as the 
# hidden state for the next time step.

### The Decoder

In [447]:
class DecoderRNN(nn.Module):
    #output_size represents the dimensionality of the 
    #decoder's output space, indicating the number of possible output tokens or
    #categories that the decoder can generate.
    
    def __init__(self, hidden_size, output_size, cell_type):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell_type):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        if cell_type == 'gru':
            output, hidden = self.gru(output, hidden)
        if cell_type == 'lstm':
            output, hidden = self.lstm(output, hidden)
        if cell_type == 'rnn':
            output, hidden = self.rnn(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### Attention Decoder

In [448]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p, max_length, cell_type):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.rnn = nn.RNN(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        if cell_type == 'gru':
            output, hidden = self.gru(output, hidden)
        if cell_type == 'lstm':
            output, hidden = self.lstm(output, hidden)
        if cell_type == 'rnn':
            output, hidden = self.rnn(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [146]:
train_data.iloc[0]

'bindhya'

## Training


### Training the Model

In [480]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden, 'rnn')
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[0]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, 'rnn')
#             loss += criterion(decoder_output, target_tensor[di])
#             decoder_input = target_tensor[di]  # Teacher forcing
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, 'rnn')
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Set the current target token as the next decoder input


    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, 'rnn')
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == 129:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [481]:
# def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
#     encoder_optimizer.zero_grad()
#     decoder_optimizer.zero_grad()

#     input_length = input_tensor.size(0)
#     target_length = target_tensor.size(0)

#     encoder_hidden = encoder.initHidden()
#     loss = 0

#     for ei in range(input_length):
#         encoder_output, encoder_hidden = encoder(
#             input_tensor[ei], encoder_hidden,'rnn')

#     decoder_input = torch.tensor([[SOS_token]])  # Start of sequence

#     decoder_hidden = encoder_hidden

#     for di in range(target_length):
#         decoder_output, decoder_hidden = decoder(
#             decoder_input, decoder_hidden, 'rnn')

#         loss += criterion(decoder_output, target_tensor[di])

#         decoder_input = target_tensor[di]  # Teacher forcing

#     loss.backward()

#     encoder_optimizer.step()
#     decoder_optimizer.step()

#     return loss.item()

In [482]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [483]:
encoder = EncoderRNN(28, 32, 'rnn').to(device)
decoder = DecoderRNN(32, 130, 'rnn').to(device)
n_iters = 4000

In [484]:
# import torch
# import torch.nn as nn
# import torch.optim as optim

# def trainIters(input_tensor, target_tensor, encoder, decoder, n_iters, batch_size, print_every=1000, plot_every=100, learning_rate=0.01):
#     import time
#     import math
#     start = time.time()
#     plot_losses = []
#     print_loss_total = 0  # Reset every print_every
#     plot_loss_total = 0  # Reset every plot_every

#     encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
#     decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
#     criterion = nn.NLLLoss()

#     # Create data loader
#     train_data = torch.utils.data.TensorDataset(input_tensor, target_tensor)
#     train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

#     total_batches = math.ceil(len(train_data) / batch_size)

#     for iter in range(1, n_iters + 1):
#         for i, (input_batch, target_batch) in enumerate(train_loader, 1):
#             input_tensor = input_batch.transpose(0, 1)  # Transpose to have batch dimension first
#             target_tensor = target_batch.transpose(0, 1)  # Transpose to have batch dimension first

#             loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)

#             print_loss_total += loss
#             plot_loss_total += loss

#             if iter % print_every == 0:
#                 print_loss_avg = print_loss_total / print_every
#                 print_loss_total = 0
#                 print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
#                                              iter, iter / n_iters * 100, print_loss_avg))

#             if iter % plot_every == 0:
#                 plot_loss_avg = plot_loss_total / plot_every
#                 plot_losses.append(plot_loss_avg)
#                 plot_loss_total = 0

#     showPlot(plot_losses)

# def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
#     encoder_optimizer.zero_grad()
#     decoder_optimizer.zero_grad()

#     input_length = input_tensor.size(0)
#     target_length = target_tensor.size(0)

#     encoder_hidden = encoder.initHidden()

#     loss = 0

#     for ei in range(input_length):
#         encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden, 'rnn')

#     decoder_input = torch.tensor([[SOS_token]])  # Start of sequence

#     decoder_hidden = encoder_hidden

#     for di in range(target_length):
#         decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden,'rnn')

#         loss += criterion(decoder_output, target_tensor[di])

#         decoder_input = target_tensor[di]  # Teacher forcing

#     loss.backward()

#     encoder_optimizer.step()
#     decoder_optimizer.step()

#     return loss.item()

# trainIters(x_train, y_train, encoder, decoder, n_iters, batch_size=64, print_every=1000, plot_every=100, learning_rate=0.01)


In [485]:
# import torch
# import torch.nn as nn
# import torch.optim as optim

# def trainIters(x_train, y_train, encoder, decoder, n_iters, batch_size, print_every=1000, plot_every=100, learning_rate=0.01):
#     import time
#     import math
#     start = time.time()
#     plot_losses = []
#     print_loss_total = 0  # Reset every print_every
#     plot_loss_total = 0  # Reset every plot_every

#     encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
#     decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
#     criterion = nn.NLLLoss()

#     # Convert data to tensors
#     x_train = torch.tensor(x_train, dtype=torch.long)
#     y_train = torch.tensor(y_train, dtype=torch.long)

#     # Create data loader
#     train_data = torch.utils.data.TensorDataset(x_train, y_train)
#     train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

#     total_batches = math.ceil(len(train_data) / batch_size)

#     for iter in range(1, n_iters + 1):
#         for i, (input_batch, target_batch) in enumerate(train_loader, 1):
#             input_tensor = input_batch.transpose(0, 1)  # Transpose to have batch dimension first
#             target_tensor = target_batch.transpose(0, 1)  # Transpose to have batch dimension first

#             loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)

#             print_loss_total += loss
#             plot_loss_total += loss

#             if iter % print_every == 0:
#                 print_loss_avg = print_loss_total / print_every
#                 print_loss_total = 0
#                 print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
#                                              iter, iter / n_iters * 100, print_loss_avg))

#             if iter % plot_every == 0:
#                 plot_loss_avg = plot_loss_total / plot_every
#                 plot_losses.append(plot_loss_avg)
#                 plot_loss_total = 0

#     showPlot(plot_losses)


In [486]:
# trainIters(x_train, y_train, encoder, decoder, n_iters, batch_size=64,
#            print_every=1000, plot_every=100, learning_rate=0.01)

In [487]:
def trainIters(x_train, y_train, encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    import time
    import math
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
#     training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
#         training_pair = training_pairs[iter - 1]
        input_tensor = x_train[iter % len(x_train)]
        target_tensor = y_train[iter % len(y_train)]


        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion, 130)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [488]:
trainIters(x_train, y_train, encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01)

ValueError: Expected input batch_size (1) to match target batch_size (0).

In [ ]:
# input_tensor=x_train_vector
# target_tensor=y_train_vector
# encoder
# decoder
# encoder_optimizer
# decoder_optimizer
# criterion
# max_length
# epochs =1
# for i in range(epochs):
#     train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length)
    